Терминалогия, используемая в задании:
* обучающая выборка - выборка, которая передается в метод fit / train;
* валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (train, valid);
* тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (train, valid, test);
* ЛБ - лидерборд, выборка assignment_2_test.csv.

In [28]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import missingno as msno
import lightgbm as lgb
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns
from scipy.stats import probplot, ks_2samp
import warnings
from tqdm import tqdm
from typing import List, Tuple

from scipy.stats import ttest_rel

from sklearn.metrics import r2_score, roc_auc_score

from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
warnings.simplefilter("ignore")
%matplotlib inline

In [33]:
train= pd.read_csv('train.csv')
l_board = pd.read_csv('test.csv')

In [34]:
train.set_index('TransactionID',inplace=True)
l_board.set_index('TransactionID',inplace=True)

In [35]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180000 entries, 2987000 to 3166999
Columns: 393 entries, isFraud to V339
dtypes: float64(376), int64(3), object(14)
memory usage: 541.1+ MB


In [37]:
l_board.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100001 entries, 3287000 to 3387000
Columns: 393 entries, isFraud to V339
dtypes: float64(376), int64(3), object(14)
memory usage: 300.6+ MB


Задание 1: сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [38]:
object_features = train.select_dtypes(include=[np.object]).columns

In [39]:
train[object_features]= train[object_features].astype('category')
l_board[object_features]= l_board[object_features].astype('category')

In [40]:
x_train,x_valid,y_train,y_valid = train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.3 ,random_state=5)

In [41]:
dtrain = lgb.Dataset(data=x_train, label=y_train,categorical_feature=object_features.to_list())
dvalid = lgb.Dataset(data=x_valid, label=y_valid,categorical_feature=object_features.to_list())

In [44]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "n_estimators": 20000,
    "n_jobs": 15,
    "seed": 5
}

In [42]:
model = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain, dvalid],
    categorical_feature=object_features.to_list(),
    early_stopping_rounds=90,
    verbose_eval=None
)

In [43]:
model.best_score, model.best_iteration

(defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9999135255845443)]),
              'valid_1': OrderedDict([('auc', 0.9602598694935909)])}),
 1335)

Качество модели на валидационной выборке 0,957  что кажеться очень хорошим результатом. При  этон на трайн почти единица. 
Что говорит, что модель сильно подстроилась по данные, но при этом разность не сильно большая меньеше 5%

Оценим качество на лидер борде

In [45]:
roc_auc_score(l_board['isFraud'], model.predict(l_board.drop('isFraud',axis=1)))

0.8549768817487624

Наблюдаем сильное отличие показателя качества на валидации и лидерборде, почти 10%

Задание 2: сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [46]:
x_valid,x_test,y_valid,y_test = train_test_split(x_valid,y_valid, test_size = 0.3 ,random_state=5)

In [47]:
x_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37800 entries, 3141998 to 3159133
Columns: 392 entries, TransactionDT to V339
dtypes: category(14), float64(376), int64(2)
memory usage: 109.8 MB


In [48]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16200 entries, 3061883 to 3090008
Columns: 392 entries, TransactionDT to V339
dtypes: category(14), float64(376), int64(2)
memory usage: 47.1 MB


In [49]:
# Скор на валидационной выборке
roc_auc_score(y_valid, model.predict(x_valid))

0.9616904373792942

In [50]:
# Скор на тестовой выборке
roc_auc_score(y_test, model.predict(x_test))

0.9569619990709486

Как видно показатель качества не сильно отличается на валидационной и на тестовой выборке и сильно преврсходит лидерборд